In [1]:
import os
print(os.getcwd())

d:\Programming\Python_code\PrinciplesOfDS_Course\Labs


In [2]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# pass in column names for each CSV as the column name is not given in the file and read them using pandas.
# You can check the column names from the readme file
#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./Data/ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('./Data/ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('./Data/ml-100k/u.item', sep='|', names=i_cols,encoding='latin-1')

In [3]:
print(users.shape)
users.head()

(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
print(ratings.shape)
ratings.head()

(100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
print(items.shape)
items.head()

(1682, 24)


,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
# read in ratings columns
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_train = pd.read_csv('./Data/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')

ratings_test = pd.read_csv('./Data/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [9]:
# finding number of unique users and movies
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]
print(n_users, n_items)

943 1682


In [14]:
#Generating user-item data matrix in order to create similarity scores
data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]
    
print(type(data_matrix))
print(data_matrix.shape)
print(data_matrix)

<class 'numpy.ndarray'>
(943, 1682)
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [40]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
# user_similarity = pd.DataFrame(data_matrix).corr(method='pearson') # DF conversion in order to use pearson correlation
# user_similarity = user_similarity.values # converting to matrix
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [41]:
print(user_similarity)
print(user_similarity.shape)

print(item_similarity)
print(item_similarity.shape)

[[0.         0.83306902 0.95254046 ... 0.85138306 0.82049212 0.60182526]
 [0.83306902 0.         0.88940868 ... 0.83851522 0.82773219 0.89420212]
 [0.95254046 0.88940868 0.         ... 0.89875744 0.86658385 0.97344413]
 ...
 [0.85138306 0.83851522 0.89875744 ... 0.         0.8983582  0.90488042]
 [0.82049212 0.82773219 0.86658385 ... 0.8983582  0.         0.81753534]
 [0.60182526 0.89420212 0.97344413 ... 0.90488042 0.81753534 0.        ]]
(943, 943)
[[0.         0.59761782 0.66975521 ... 1.         0.95281693 0.95281693]
 [0.59761782 0.         0.72693082 ... 1.         0.92170064 0.92170064]
 [0.66975521 0.72693082 0.         ... 1.         1.         0.90312495]
 ...
 [1.         1.         1.         ... 0.         1.         1.        ]
 [0.95281693 0.92170064 1.         ... 1.         0.         1.        ]
 [0.95281693 0.92170064 0.90312495 ... 1.         1.         0.        ]]
(1682, 1682)


In [42]:
# checking summary stats on the matrices: 
print('USER -- min:',np.min(user_similarity), "max:", np.max(user_similarity)) 
print('ITEM -- min:',np.min(item_similarity), "max:", np.max(item_similarity)) # I thought cosine similarity could be negative.. hm.. may need to learn more on that 

USER -- min: 0.0 max: 1.0
ITEM -- min: 0.0 max: 1.0


In [37]:
def predict(ratings, similarity, data_type='user'):
    
    if data_type == 'user':
        mean_user_rating = ratings.mean(axis=1).reshape(-1,1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        #print(type(mean_user_rating), mean_user_rating, mean_user_rating.shape)
        ratings_diff = (ratings - mean_user_rating)
        pred = mean_user_rating + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    
    elif data_type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    
    return pred

In [43]:
print(user_similarity.shape, data_matrix.shape)
print(item_similarity.shape, data_matrix.shape)

(943, 943) (943, 1682)
(1682, 1682) (943, 1682)


In [44]:
user_prediction = predict(data_matrix, user_similarity, data_type='user')
item_prediction = predict(data_matrix, item_similarity, data_type='item')

In [45]:
print('user_prediction', user_prediction)
print('item_prediction', item_prediction)

user_prediction [[ 2.06532606  0.73430275  0.62992381 ...  0.39359041  0.39304874
   0.3927712 ]
 [ 1.76308836  0.38404019  0.19617889 ... -0.08837789 -0.0869183
  -0.08671183]
 [ 1.79590398  0.32904733  0.15882885 ... -0.13699223 -0.13496852
  -0.13476488]
 ...
 [ 1.59151513  0.27526889  0.10219534 ... -0.16735162 -0.16657451
  -0.16641377]
 [ 1.81036267  0.40479877  0.27545013 ... -0.00907358 -0.00846587
  -0.00804858]
 [ 1.8384313   0.47964837  0.38496292 ...  0.14686675  0.14629808
   0.14641455]]
item_prediction [[0.44627765 0.475473   0.50593755 ... 0.58815455 0.5731069  0.56669645]
 [0.10854432 0.13295661 0.12558851 ... 0.13445801 0.13657587 0.13711081]
 [0.08568497 0.09169006 0.08764343 ... 0.08465892 0.08976784 0.09084451]
 ...
 [0.03230047 0.0450241  0.04292449 ... 0.05302764 0.0519099  0.05228033]
 [0.15777917 0.17409459 0.18900003 ... 0.19979296 0.19739388 0.20003117]
 [0.24767207 0.24489212 0.28263031 ... 0.34410424 0.33051406 0.33102478]]
